# Let's train some models to deploy...

* **Want:** [`SavedModel`](https://www.tensorflow.org/guide/saved_model) format to upload to Google Storage (GS).
* **Data:** Different slices of [Food101 dataset](https://www.kaggle.com/dansbecker/food-101).
* **Model(s):** [EfficientNetB0](https://www.tensorflow.org/api_docs/python/tf/keras/applications/EfficientNetB0) backbone's with different output layers (e.g. 10 classes, 11 classes, 12 classes).


In [9]:
# Are we using a GPU? 
!nvidia-smi

Fri Jul  9 16:37:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 466.77       Driver Version: 466.77       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:2B:00.0 Off |                  N/A |
| N/A   59C    P8    N/A /  N/A |     64MiB /  2048MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Setup helper functions

In [11]:
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Unzip the downloaded file
def unzip_data(filename):
  """
  Utility function to unzip a zipped file.
  """
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

In [12]:
# Setup data inputs
IMG_SIZE = (224, 224)

def create_data_loaders(train_dir, test_dir, image_size=IMG_SIZE):
  """
  Creates a training and test image BatchDataset from train_dir and test_dir.
  """
  train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                  label_mode="categorical",
                                                                  image_size=image_size)
  # Note: the test data is the same as the previous experiment, we could
  # skip creating this, but we'll leave this here to practice.
  test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                  label_mode="categorical",
                                                                  image_size=image_size)
  
  return train_data, test_data

In [13]:
# Create a data augmentation stage with horizontal flipping, rotations, zooms
data_augmentation = keras.Sequential([
  preprocessing.RandomFlip("horizontal"),
  preprocessing.RandomRotation(0.2),
  preprocessing.RandomZoom(0.2),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),
  # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNetB0
], name ="data_augmentation")

In [14]:
# Setup input shape and base model, freezing the base model layers
INPUT_SHAPE = (224, 224, 3)
BASE_MODEL = tf.keras.applications.EfficientNetB0(include_top=False)

def create_model(input_shape=INPUT_SHAPE, base_model=BASE_MODEL, num_classes=10):
  # Fine-tune?
  base_model.trainable = False

  # Create input layer
  inputs = layers.Input(shape=input_shape, name="input_layer")

  # Add in data augmentation Sequential model as a layer
  x = data_augmentation(inputs)

  # Give base_model inputs (after augmentation) and don't train it
  x = base_model(x, training=False)

  # Pool output features of base model
  x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

  # Put a dense layer on as the output
  outputs = layers.Dense(num_classes, activation="softmax", name="output_layer")(x)

  # Make a model with inputs and outputs
  model = keras.Model(inputs, outputs)

  # Compile the model
  model.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
  
  return model

16711680/16705208 [==============================] - 1s 0us/step


In [15]:
# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224, scale=False):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  # Rescale the image (get all values between 0 and 1)
  if scale:
    return img/255.
  else:
    return img

## Download data

In [24]:
# Get data
import zipfile

# Download data (10 class subset of Food101 - https://www.kaggle.com/dansbecker/food-101)
# Already formatted in standard image classification directory style
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

unzip_data("10_food_classes_all_datas.zip")

In [21]:
# How many images in each folder?
import os

# Walk through data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_all_data"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_all_data'.
There are 10 directories and 0 images in '10_food_classes_all_data\test'.
There are 0 directories and 250 images in '10_food_classes_all_data\test\chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_all_data\test\chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_all_data\test\fried_rice'.
There are 0 directories and 250 images in '10_food_classes_all_data\test\grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_all_data\test\hamburger'.
There are 0 directories and 250 images in '10_food_classes_all_data\test\ice_cream'.
There are 0 directories and 250 images in '10_food_classes_all_data\test\pizza'.
There are 0 directories and 250 images in '10_food_classes_all_data\test\ramen'.
There are 0 directories and 250 images in '10_food_classes_all_data\test\steak'.
There are 0 directories and 250 images in '10_food_classes_all_data\test\sushi'.
There are 

In [9]:
# Check the file in 10_food_classes_10_percent
!ls -la 10_food_classes_all_data

total 28
drwxr-xr-x  4 root root 4096 Feb 14 05:54 .
drwxr-xr-x  1 root root 4096 Feb 14 05:54 ..
-rw-r--r--  1 root root 8196 Feb 14 05:54 .DS_Store
drwxr-xr-x 12 root root 4096 Feb 14 05:54 test
drwxr-xr-x 12 root root 4096 Feb 14 05:54 train


In [25]:
# Create tensorboard callback (functionized because need to create a new one for each model)
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

* `dir_name` is the overall logs directory
* `experiment_name` is the particular experiment
* `current_timestamp` is the time the experiment started based on Python's [`datetime.datetime().now()`](https://docs.python.org/3/library/datetime.html#datetime.datetime.now)




## Model (10 classes)

In [26]:
# Create BatchDataset
train_data, test_data = create_data_loaders(train_dir="10_food_classes_all_data/train/",
                                            test_dir="10_food_classes_all_data/test/")

Found 7500 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [27]:
# What size is our data?
train_data

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>

In [28]:
# Create model
model_1 = create_model(num_classes=len(train_data.class_names))

# Fit the model
history_1_percent = model_1.fit(train_data,
                    epochs=5,
                    steps_per_epoch=len(train_data),
                    validation_data=test_data,
                    validation_steps=int(0.25 * len(test_data)), # validate for less steps
                    # Track model training logs
                    callbacks=[create_tensorboard_callback("transfer_learning", "all_data_aug")])

Saving TensorBoard log files to: transfer_learning/all_data_aug/20210709-164936
C:\Python38\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
Epoch 1/5
235/235 [==============================] - 278s 1s/step - loss: 1.0708 - accuracy: 0.6880 - val_loss: 0.5016 - val_accuracy: 0.8503
Epoch 2/5
235/235 [==============================] - 244s 1s/step - loss: 0.6838 - accuracy: 0.7827 - val_loss: 0.4203 - val_accuracy: 0.8684
Epoch 3/5
235/235 [==============================] - 253s 1s/step - loss: 0.6133 - accuracy: 0.8096 - val_loss: 0.3868 - val_accuracy: 0.8799
Epoch 4/5
235/235 [==============================] - 265s 1s/step - loss: 0.5654 - accuracy: 0.8187 - val_loss: 0.3834 - val_accuracy: 0.8783
Epoch 5/5
235/235 [==========

In [ ]:
# Get an image Tensor
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-pizza-dad.jpeg

--2021-02-10 23:18:11--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-pizza-dad.jpeg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874848 (2.7M) [image/jpeg]
Saving to: ‘03-pizza-dad.jpeg’

03-pizza-dad.jpeg   100%[===================>]   2.74M  --.-KB/s    in 0.1s    

2021-02-10 23:18:11 (27.7 MB/s) - ‘03-pizza-dad.jpeg’ saved [2874848/2874848]



In [30]:
# Classes our model is trained on
class_names = train_data.class_names
class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [31]:
# Preprocess image
pizza_img = load_and_prep_image("03-pizza-dad.jpeg")
pizza_img

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[ 73.625,  76.75 ,  67.125],
        [114.   , 122.   , 101.   ],
        [146.875, 151.875, 129.875],
        ...,
        [ 14.5  ,  17.5  ,  10.5  ],
        [ 14.25 ,  19.25 ,  12.25 ],
        [ 19.75 ,  22.75 ,  15.75 ]],

       [[239.125, 243.625, 246.125],
        [225.375, 232.125, 234.875],
        [240.   , 245.   , 244.5  ],
        ...,
        [ 11.   ,  14.   ,   7.   ],
        [ 20.   ,  23.   ,  16.   ],
        [ 20.875,  25.875,  18.875]],

       [[ 32.5  ,  34.5  ,  31.5  ],
        [ 44.625,  44.5  ,  42.375],
        [ 33.   ,  38.   ,  34.   ],
        ...,
        [  8.75 ,  13.25 ,   6.25 ],
        [ 14.875,  17.875,  10.875],
        [ 13.625,  20.625,  12.625]],

       ...,

       [[ 61.875,  40.875,  19.875],
        [ 60.   ,  42.   ,  22.   ],
        [ 61.   ,  43.   ,  21.   ],
        ...,
        [134.5  ,  96.125,  60.75 ],
        [104.875,  69.375,  43.125],
        [106.25 ,  75.2

In [32]:
# Make predictions
pizza_expanded = tf.expand_dims(pizza_img, axis=0) # expand image dimensions (224, 224, 3) -> (1, 224, 224, 3)
pred = model_1.predict(pizza_expanded)
pred

array([[1.1428705e-02, 1.1287200e-04, 4.7732206e-04, 1.0897476e-04,
        4.2807296e-06, 1.0963952e-06, 9.8670828e-01, 1.4884207e-04,
        1.3133651e-04, 8.7829010e-04]], dtype=float32)

In [33]:
# Check the predicted class
class_names[tf.argmax(pred[0])]

'pizza'

## Save and upload Model to Google Storage

Upload the model to Google Storage using the following guide: https://cloud.google.com/storage/docs/uploading-objects#gsutil 

In [34]:
# Save model_1
model_1.save("efficientnet_model_1_10_classes")

C:\Python38\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
INFO:tensorflow:Assets written to: efficientnet_model_1_10_classes\assets


In [36]:
# Copy model to bucket - https://cloud.google.com/storage/docs/uploading-objects#gsutil 
# Use "-r" for folders (r stands for recursive)
!gsutil cp -r efficientnet_model_1_10_classes gs://patrick_ml_deployment_bucket

'gsutil' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [ ]:
# Copy model to bucket
!gsutil cp -r efficientnet_model_1_10_classes gs://cs329s_live_bucket_creation

Copying file://efficientnet_model_1_10_classes/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://efficientnet_model_1_10_classes/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://efficientnet_model_1_10_classes/variables/variables.index [Content-Type=application/octet-stream]...
- [3 files][ 22.1 MiB/ 22.1 MiB]                                                
Operation completed over 3 objects/22.1 MiB.                                     
